In [14]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -uq "/content/drive/My Drive/Dataset/Data1.zip"

In [0]:
from keras.preprocessing.image import ImageDataGenerator


In [0]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [0]:
test_datagen = ImageDataGenerator(rescale=1. / 255)
valid_datagen= ImageDataGenerator(rescale=1. / 255)

In [0]:
train_data_dir="/content/Data/Train"
validation_data_dir="/content/Data/Valid"
img_height, img_width=227,227
batch_size=16

In [20]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 6049 images belonging to 9 classes.


In [0]:
nb_train_samples=6049

In [22]:
validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

Found 1397 images belonging to 9 classes.


In [23]:
test_generator = test_datagen.flow_from_directory(
    directory=r"/content/Data/Test",
    target_size=(224, 224),
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 1863 images belonging to 1 classes.


In [0]:
nb_validation_samples=1397
nb_test_samples=1863

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, CSVLogger
import numpy as np
np.random.seed(9)

In [0]:
#Instantiate an empty model
model = Sequential()

In [0]:
# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

In [28]:
# Passing it to a Fully Connected layer
# model.add(BatchNormalization())
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(227*227*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.5))

# 2nd Fully Connected Layer
model.add(BatchNormalization())
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.5))

# 3rd Fully Connected Layer
model.add(BatchNormalization())
model.add(Dense(1000))
# model.add(Dense(9))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(9))
model.add(Activation('softmax'))

model.summary()

# Compile the model
from keras import optimizers
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=sgd, metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='best_model_pretrain.hdf5', verbose=1, save_best_only=True)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_10 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 23, 23, 256)       614656    
_________________________________________________________________
activation_11 (Activation)   (None, 23, 23, 256)       0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 9, 9, 384)        

In [0]:
def unfreeze_layer_onwards(model, layer_name):
    '''
        This layer unfreezes all layers beyond layer_name
    '''
    trainable = False
    for layer in model.layers:
        try:
            if layer.name == layer_name:
                trainable = True
            layer.trainable = trainable
        except:
            continue
    
    return model
    

In [0]:
def append_history(history, h):
     '''
	This function appends the statistics over epochs
     '''
     try:
       history.history['loss'] = history.history['loss'] + h.history['loss']
       history.history['val_loss'] = history.history['val_loss'] + h.history['val_loss']
       history.history['acc'] = history.history['acc'] + h.history['acc']
       history.history['val_acc'] = history.history['val_acc'] + h.history['val_acc']
     except:
       history = h
                
     return history

In [0]:
layers = ['dense_9','dense_8','dense_7','conv2d_15','conv2d_14','conv2d_13','conv2d_12','conv2d_11']
epochs = [10,10,10,10,10,10,10,10]
lr = [1e-2,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3]
from keras.optimizers import SGD
history_finetune = []

for i,layer in enumerate(layers):
        
    alexnet = unfreeze_layer_onwards(model,layer)    
   
    sgd = SGD(lr=lr[i], decay=1e-6, momentum=0.9, nesterov=True)
    alexnet.compile(loss='mse',
              optimizer=sgd,
              metrics=['accuracy'])
    
    for epoch in range(epochs[i]):    
        h = alexnet.fit_generator(train_generator,
                    steps_per_epoch = nb_train_samples //batch_size,
                    validation_data=validation_generator,
                    validation_steps= nb_validation_samples //batch_size,
                    epochs=90,
                    verbose=1,
                    callbacks=[checkpointer])
        
        history_finetune = append_history(history_finetune,h)

In [0]:
# # Train Model
# history = model.fit(
#         train_generator,
#         epochs=150,
#         validation_data = validation_generator, 
#         callbacks=[checkpointer]
#         )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/150


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 2. Skipping tag 41487
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 41988
  " Skipping tag %s" % (size, len(data), tag))







379/379 [==============================] - 146s 385ms/step - loss: 2.9285 - acc: 0.1435 - val_loss: 2.2272 - val_acc: 0.1475

Epoch 00001: val_loss improved from inf to 2.22717, saving model to best_model_pretrain.hdf5
Epoch 2/150
379/379 [==============================] - 137s 361ms/step - loss: 2.6790 - acc: 0.1557 - val_loss: 2.4596 - val_acc: 0.1976

Epoch 00002: val_loss did not improve from 2.22717
Epoch 3/150
379/379 [==============================] - 138s 363ms/step - loss: 2.4996 - acc: 0.1616 - val_loss: 2.0564 - val_acc: 0.2119

Epoch 00003: val_loss improved from 2.22717 to 2.05640, saving model to best_model_pretrain.hdf5
Epoch 4/150
379/379 [==============================] - 137s 361ms/step - loss: 2.4031 - acc: 0.1618 - val_loss: 2.2721 - val_acc: 0.1224

Epoch 00004: val_loss did not improve from 2.05640
Epoch 5/150
379/379 [==============================] - 137s 362ms/step - loss: 2.3120 - acc: 0.1694 - val_loss: 2.2715 - val_acc: 0.1496

Epoch 00005: val_loss did

load alexnet
